In [ ]:
# librerias

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
import warnings
import matplotlib.pyplot as plt

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# init
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
scopus = pd.read_csv("https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/scopus-abstracts.csv")
scopus['Abstract'].head()

0    Mobility is one of the fundamental requirement...
1    The recent rise of the political extremism in ...
2    The power of the press to shape the informatio...
3    Identifying influential nodes in a network is ...
4    To complement traditional dietary surveys, whi...
Name: Abstract, dtype: object

In [ ]:
# limpieza

datos = scopus.Abstract.values.tolist()

# Remove Emails
datos = [re.sub(r'\S*@\S*\s?', '', sent) for sent in datos]
# Remove new line characters
datos = [re.sub(r'\s+', ' ', sent) for sent in datos]
# Remove distracting single quotes
datos = [re.sub(r"\'", "", sent) for sent in datos]
pprint(datos[:1])

['Mobility is one of the fundamental requirements of human life with '
 'significant societal impacts including productivity, economy, social '
 'wellbeing, adaptation to a changing climate, and so on. Although human '
 'movements follow specific patterns during normal periods, there are limited '
 'studies on how such patterns change due to extreme events. To quantify the '
 'impacts of an extreme event to human movements, we introduce the concept of '
 'mobility resilience which is defined as the ability of a mobility system to '
 'manage shocks and return to a steady state in response to an extreme event. '
 'We present a method to detect extreme events from geo-located movement data '
 'and to measure mobility resilience and transient loss of resilience due to '
 'those events. Applying this method, we measure resilience metrics from '
 'geo-located social media data for multiple types of disasters occurred all '
 'over the world. Quantifying mobility resilience may help us to asse

In [ ]:
 #Convert a document into a list of tokens
def sent_to_words(sentences):
  for sentence in sentences:
      yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(datos))  #datos tokenizados
print(data_words[:1])

[['mobility', 'is', 'one', 'of', 'the', 'fundamental', 'requirements', 'of', 'human', 'life', 'with', 'significant', 'societal', 'impacts', 'including', 'productivity', 'economy', 'social', 'wellbeing', 'adaptation', 'to', 'changing', 'climate', 'and', 'so', 'on', 'although', 'human', 'movements', 'follow', 'specific', 'patterns', 'during', 'normal', 'periods', 'there', 'are', 'limited', 'studies', 'on', 'how', 'such', 'patterns', 'change', 'due', 'to', 'extreme', 'events', 'to', 'quantify', 'the', 'impacts', 'of', 'an', 'extreme', 'event', 'to', 'human', 'movements', 'we', 'introduce', 'the', 'concept', 'of', 'mobility', 'resilience', 'which', 'is', 'defined', 'as', 'the', 'ability', 'of', 'mobility', 'system', 'to', 'manage', 'shocks', 'and', 'return', 'to', 'steady', 'state', 'in', 'response', 'to', 'an', 'extreme', 'event', 'we', 'present', 'method', 'to', 'detect', 'extreme', 'events', 'from', 'geo', 'located', 'movement', 'data', 'and', 'to', 'measure', 'mobility', 'resilience', 

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

  # Initialize spacy ‘en’ model, keeping only tagger component (for efficiency)
# Run in terminal: python -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])  
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']) #select noun and verb
print(data_lemmatized[:2])

['mobility fundamental requirement human life significant societal impact include productivity economy social wellbeing adaptation change climate so on human movement follow specific pattern normal period limited study how such pattern change extreme event quantify impact extreme event human movement introduce concept mobility resilience define ability mobility system manage shock return steady state response extreme event present method detect extreme event locate movement datum measure mobility resilience transient loss resilience event apply method measure resilience metric locate social medium datum multiple type disaster occur all world quantifying mobility resilience may help assess high order economic impact extreme event guide policy develop resilient infrastructure as well nation overall disaster resilience strategie author', 'recent rise political extremism western country spur renew interest psychological moral appeal political extremism empirical support psychological expla

In [ ]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,# minimum reqd occurences of a word 
                             stop_words='english', # remove stop words
                             lowercase=True, # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,    # max number of uniq words    
                            )
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [ ]:
terms = vectorizer.get_feature_names()
len(terms)

1905

In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,           # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',  # learning method
                                      random_state=100,          # Random state
                                      #batch_size=128,           # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs      
                                     )

lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [ ]:
 LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)

In [ ]:
#Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
pprint(lda_model.get_params())

Perplexity:  908.1986511905833
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [ ]:
# Define Search Param
search_params = {'n_components': [6, 7, 8, 9, 10, 15, 20], 'learning_decay': [.5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid= search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(learning_method='online',
                                                 learning_offset=50.0,
                                                 max_iter=5, random_state=0),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [6, 7, 8, 9, 10, 15, 20]})

In [ ]:
GridSearchCV(estimator=LatentDirichletAllocation(learning_method='online',
                                                 learning_offset=50.0,
                                                 max_iter=5, random_state=0),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [5, 15, 20]})

GridSearchCV(estimator=LatentDirichletAllocation(learning_method='online',
                                                 learning_offset=50.0,
                                                 max_iter=5, random_state=0),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [5, 15, 20]})

In [ ]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 6}
Best Log Likelihood Score:  -219062.29150914232
Model Perplexity:  933.907220494982


In [ ]:
 # Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(datos))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
  color = 'green' if val > .1 else 'black'
  return 'color: {col}'.format(col=color)
def make_bold(val):
  weight = 700 if val > .1 else 400
  return 'font-weight: {weight}'.format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.000000,0.000000,0.000000,0.540000,0.000000,0.460000,3
Doc1,0.000000,0.000000,0.000000,0.800000,0.190000,0.000000,3
Doc2,0.000000,0.000000,0.600000,0.390000,0.000000,0.000000,2
Doc3,0.000000,0.000000,0.000000,0.990000,0.000000,0.000000,3
Doc4,0.000000,0.000000,0.210000,0.550000,0.230000,0.000000,3
Doc5,0.000000,0.000000,0.020000,0.820000,0.150000,0.000000,3
Doc6,0.000000,0.000000,0.390000,0.600000,0.000000,0.000000,3
Doc7,0.000000,0.000000,0.000000,0.780000,0.210000,0.000000,3
Doc8,0.000000,0.190000,0.000000,0.800000,0.000000,0.000000,3
Doc9,0.000000,0.000000,0.000000,0.990000,0.000000,0.000000,3


# Matriz palabra clave

In [ ]:
 # Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

,ability,able,absence,absolute,abstract,abstraction,academia,academic,accelerate,accept,acceptance,access,accessibility,accessible,accident,accommodate,accompany,accomplish,accord,accordingly,account,accumulate,accuracy,accurate,accurately,achieve,acquire,acquisition,act,action,actionable,active,activity,actual,actually,adapt,adaptation,adaptive,add,addition,...,visualisation,visualization,visualize,vital,vocabulary,volume,vulnerable,walk,want,warehouse,warning,waste,water,wave,way,weak,weakness,weather,web,website,week,weight,western,white,wide,widely,widespread,window,word,work,worker,workflow,workload,world,worldwide,write,xml,year,yield,zone
Topic0,0.662198,0.916299,0.932966,13.444765,0.364849,0.376486,0.385337,0.369579,0.703496,0.519450,0.404341,0.569669,0.403032,0.451327,0.425309,0.333199,0.426864,0.403651,0.359385,0.321983,0.900006,0.396943,0.899004,0.364097,0.504799,0.652054,2.119789,1.643722,0.345733,0.408878,0.365886,0.628205,1.815095,0.730893,0.347574,0.375688,0.339949,0.386639,0.645224,1.346060,...,0.382924,0.448248,0.354853,0.349171,0.367941,0.510953,0.346555,0.326288,0.360378,0.455017,0.380259,0.345264,0.413048,1.495574,0.655168,0.340970,0.442045,1.233541,0.379095,0.825497,0.497749,0.420984,1.140349,0.483705,0.452021,0.390972,0.368708,0.361899,0.938191,0.532829,0.383669,0.684909,0.372453,0.607160,0.928063,0.428145,0.366792,2.951260,1.083607,2.292307
Topic1,10.101933,4.499002,3.932916,1.901586,0.689568,0.502239,0.343042,0.415561,1.228006,0.952564,0.559556,0.660546,1.101038,0.452940,0.374689,1.204577,0.475007,1.588920,7.044328,0.683758,8.122646,0.386989,40.695482,3.840164,3.240327,16.480204,0.987961,5.257080,1.683505,0.913888,1.068120,0.537971,1.617128,3.374438,1.154293,1.029357,1.285100,1.120767,4.917791,10.878085,...,0.883153,0.998072,0.639681,2.659488,0.350930,1.276361,0.815638,4.920700,0.508592,18.997866,1.853283,0.977582,0.518111,0.390015,5.981087,1.015508,0.970743,0.445082,1.013201,0.732585,0.512927,16.640265,0.408581,0.382788,3.214784,3.506108,0.565646,1.734190,0.722726,13.983111,0.584622,0.392006,2.631433,17.290788,1.140071,0.736274,0.591292,3.036672,9.155214,1.356023
Topic2,0.383742,0.407520,0.390360,0.374267,16.905521,0.369641,0.343983,0.411176,0.387506,0.389870,0.604591,5.009791,2.081563,0.406984,0.477312,0.378524,0.758311,0.355629,0.498957,0.377794,0.477551,0.372675,0.500328,0.490258,0.391358,0.557052,0.355968,0.335098,0.389833,0.337391,0.341723,0.487417,0.628925,0.478035,0.356450,0.494131,0.392039,0.351880,0.414484,0.611685,...,0.758536,0.441246,1.341973,0.508762,0.336545,0.960563,0.416681,2.011028,1.032370,0.360768,0.372546,0.368697,0.508519,0.382809,0.706831,0.355950,1.059096,0.369368,0.505811,0.345504,0.389745,0.415820,0.419609,0.566077,1.282977,0.447963,0.515093,0.340204,0.402506,1.616166,0.366798,0.344727,0.371792,1.040206,0.427422,0.441515,0.409985,2.071125,2.685277,1.938052
Topic3,20.084111,25.976564,3.851505,3.755137,0.528543,0.435664,1.114002,11.140335,2.579470,3.874969,8.161350,36.505256,31.611370,9.028266,37.778798,3.129246,2.108716,0.490654,8.008631,1.086802,23.574536,2.047501,18.144319,6.066169,8.504277,20.117658,5.179094,0.534236,5.123146,18.578137,0.704182,13.522946,104.663442,8.171860,1.861085,10.503467,4.840449,2.160816,7.326430,25.212121,...,1.997981,5.014872,1.342280,4.607254,0.394470,13.267825,4.609718,19.786939,1.616876,0.457159,4.446688,1.578655,2.305083,5.648529,32.874211,8.834858,0.479232,1.171804,2.071605,10.876995,12.623725,13.269178,9.179133,12.592697,25.426829,12.849346,2.907293,8.984419,9.368143,82.140761,10.991676,0.383612,0.466870,40.260923,10.788223,2.732381,0.357825,63.757717,11.252797,19.002077
Topic4,32.746433,54.082415,5.811774,1.453171,13.824820,10.702620,7.117400,28.541378,8.277539,12.047057,6.575236,119.413002,13.082585,20.767625,0.993188,8.711582,6.499585,15.201118,35.285882,13.863227,26.111159,8.495622,83.246812,55.655956,8.641496,93.149167,24.985014,19.204931,12.255069,21.261033,9.009614,12.433528,77.380927,12.984452,7.084653,25.257470,6.356275,21.602010,29.997

# n palabras clave

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,geomagnetic,observatory,magnetic,field,magnetometer,record,absolute,measurement,observation,baseline,variation,solar,geophysical,reading,station
Topic 1,model,method,propose,use,graph,algorithm,result,paper,base,approach,performance,product,large,optimal,value
Topic 2,segregation,food,income,shape,variable,bus,method,study,available,abstract,use,correlation,news,route,factor
Topic 3,use,study,urban,network,model,social,city,result,analysis,author,different,datum,base,spatial,measure
Topic 4,datum,use,base,paper,information,propose,method,research,data,model,big,result,approach,algorithm,analysis
Topic 5,crowd,disaster,crime,regional,planning,recovery,impact,datum,video,use,base,time,event,natural,analysis


In [ ]:
df_document_topic['dominant_topic'].value_counts()

4    1114
3     496
1     228
2      33
0      22
5       9
Name: dominant_topic, dtype: int64